In [1]:
import os
os.chdir('/Users/francopiccolo/GitHub/learn-to-rank')

In [2]:
import lightgbm as lgb

In [3]:
import pandas as pd
X_train = pd.read_pickle('./data/X_train.pkl')
X_val = pd.read_pickle('./data/X_val.pkl')

y_train = pd.read_pickle('./data/y_train.pkl')
y_val = pd.read_pickle('./data/y_val.pkl')

train_query_sizes = pd.read_pickle('./data/train_query_sizes.pkl')
val_query_sizes = pd.read_pickle('./data/val_query_sizes.pkl')

In [4]:
gbm = lgb.LGBMRanker(random_state=1)
gbm.fit(X_train, y_train, group=train_query_sizes,
        eval_set=[(X_val, y_val)], eval_group=[val_query_sizes], eval_at = [1, 3, 5, 10],
        early_stopping_rounds=10, verbose=True, 
        callbacks=[lgb.reset_parameter(learning_rate=lambda x: 0.95 ** x * 0.1)])

[1]	valid_0's ndcg@1: 0.238506	valid_0's ndcg@3: 0.383332	valid_0's ndcg@5: 0.452466	valid_0's ndcg@10: 0.525635
Training until validation scores don't improve for 10 rounds
[2]	valid_0's ndcg@1: 0.23985	valid_0's ndcg@3: 0.382761	valid_0's ndcg@5: 0.452402	valid_0's ndcg@10: 0.526926
[3]	valid_0's ndcg@1: 0.242045	valid_0's ndcg@3: 0.386567	valid_0's ndcg@5: 0.454786	valid_0's ndcg@10: 0.5292
[4]	valid_0's ndcg@1: 0.244828	valid_0's ndcg@3: 0.388128	valid_0's ndcg@5: 0.456775	valid_0's ndcg@10: 0.531296
[5]	valid_0's ndcg@1: 0.244794	valid_0's ndcg@3: 0.38941	valid_0's ndcg@5: 0.457301	valid_0's ndcg@10: 0.53161
[6]	valid_0's ndcg@1: 0.247422	valid_0's ndcg@3: 0.391284	valid_0's ndcg@5: 0.46062	valid_0's ndcg@10: 0.534212
[7]	valid_0's ndcg@1: 0.249448	valid_0's ndcg@3: 0.394309	valid_0's ndcg@5: 0.462685	valid_0's ndcg@10: 0.535855
[8]	valid_0's ndcg@1: 0.251035	valid_0's ndcg@3: 0.396445	valid_0's ndcg@5: 0.464791	valid_0's ndcg@10: 0.537354
[9]	valid_0's ndcg@1: 0.251758	valid_0's 

LGBMRanker(random_state=1)

In [9]:
pd.DataFrame(gbm.best_score_)

,valid_0
ndcg@1,0.265604
ndcg@10,0.550946
ndcg@3,0.414681
ndcg@5,0.482432


In [10]:
gbm.booster_.save_model('./data/model.txt')

In [7]:
# baseline model and param optimization
# wont go into it

In [8]:
# lightgbm api
# train_ds = lgb.Dataset('./data/train_ds.bin')
# val_ds = lgb.Dataset('./data/val_ds.bin')
# param = {'objective': 'lambdarank',
#          'metric': 'ndcg'}
# bst = lgb.train(param, train_ds, valid_sets=[val_ds], early_stopping_rounds=5)
# bst.save_model('model.txt', num_iteration=bst.best_iteration)
# bst = lgb.Booster(model_file='model.txt')
# ypred = bst.predict(test_ds) # num_iteration=bst.best_iteration